In [7]:
from mxnet import nd, gpu, cpu

ctx = gpu(1)

### 자료 불러오기
- 불러온 자료는 리스트입니다.
- 리스트의 원소는 (프레임 수, 17, 2) 형상을 한 mxnet ndarray 입니다

### 우리가 찍은 동영상 불러오기

In [8]:
import pickle
with open('./data_pkl/our_data_five.pkl', 'rb') as f :
    x_data = pickle.load(f)
    y_data = pickle.load(f)

### 제공받은 이미지 불러오기

In [9]:
import pickle
with open('./data_pkl/img_data_five.pkl', 'rb') as f :
    x_data += pickle.load(f)
    y_data += pickle.load(f)
    
print(len(x_data), len(y_data))
print(x_data[1000].shape, y_data[1000].shape)

2122 2122
(26, 17, 2) (1,)


### 자료 전처리
- 자료의 시계열 길이를 맞춰줍니다. 94 프레임. `padded`
- (17, 2) 형상의 관절점을 34개의 좌표로 펼칩니다. `flat`

In [10]:
max_len = 94
padded = [nd.concat(x, nd.zeros((max_len-x.shape[0], 17, 2)).as_in_context(ctx), dim=0) for x in x_data]
flat = [p.reshape(-1, 34) for p in padded]

- 리스트는 이제 필요 없습니다. ndarray로 바꿉니다

In [11]:
x_data = nd.stack(*flat)
y_data = nd.stack(*[y for y in y_data])
print(x_data.shape, y_data.shape)

(2122, 94, 34) (2122, 1)


- 학습 자료와 검증 자료로 나눕니다

In [12]:
import numpy as np

num_data = len(x_data)
choice_t = np.random.choice(num_data, (int)(num_data*0.80), replace=False)
choice_v = [n for n in range(num_data) if not n in choice_t]

x_train, y_train = x_data[choice_t], y_data[choice_t]
x_valid, y_valid = x_data[choice_v], y_data[choice_v]
num_train, num_valid = len(x_train), len(x_valid)

print(x_train.shape, x_valid.shape, num_data)

(1697, 94, 34) (425, 94, 34) 2122


### 자료 분포 확인

In [17]:
dist = []
for i in range(5) :
    dist.append((y_data == i).sum())
        
[d / sum(dist) * 100 for d in dist]

[
 [19.227144]
 <NDArray 1 @gpu(1)>, 
 [20.546654]
 <NDArray 1 @gpu(1)>, 
 [20.735155]
 <NDArray 1 @gpu(1)>, 
 [14.985863]
 <NDArray 1 @gpu(1)>, 
 [24.505182]
 <NDArray 1 @gpu(1)>]

### 학습

In [44]:
from mxnet import gluon, init
from mxnet import autograd
from mxnet.gluon import nn, rnn
import mxnet as mx
from SkeletonARModel import SkeletonARModel

- 모델을 생성합니다
- 가중치를 초기화합니다.
- `SkeletonAR.params` 에는 5중 분류기의 학습된 가중치가 저장되어 있습니다.

In [47]:
model = SkeletonARModel()
model.collect_params().initialize(init.Xavier(), force_reinit=True, ctx=ctx)
model.load_parameters("./model_params/SkeletonAR.params", ignore_extra=True, allow_missing=True, ctx=ctx)

In [48]:
lr = 0.005
acc = mx.metric.Accuracy()
Loss = gluon.loss.SoftmaxCrossEntropyLoss(sparse_label=True)
trainer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': lr})

for epoch in range(10000):
    
    for i in range(4) :
        choice = np.random.choice(num_train, int(num_train * 0.25), replace=False)
        
        with autograd.record() :
            y_hat = model(x_train[choice])
            loss = Loss(y_hat, y_train[choice])
        loss.backward()
        trainer.step(batch_size=len(choice))
        
    print("\n******Epoch {%d} ******" % epoch)
    print("**loss > {}".format(nd.sum(loss / len(choice)).asscalar()))
        
    if epoch % 20 == 0 :
        try :
            acc.update(preds=y_hat, labels=y_train[choice])
            print("**\t train_acc >> {}".format(acc.get()))

            valid_y_hat = model(x_valid)
            valid_loss = Loss(valid_y_hat, y_valid)
            print("**\t valid loss >> {}".format(nd.sum(valid_loss / num_valid).asscalar()))

            acc.update(preds=valid_y_hat, labels=y_valid)
            print("**\t valid_acc >> {}, ".format(acc.get()))

            print("**\t predicts >> ", nd.argmax(valid_y_hat, axis=1))

        except KeyboardInterrupt :
            break
        except :
            print("errors")    


******Epoch {0} ******
**loss > 0.12645193934440613
**	 train_acc >> ('accuracy', 0.9693396226415094)
**	 valid loss >> 0.15134508907794952
**	 valid_acc >> ('accuracy', 0.9658421672555948), 
**	 predicts >>  
[0. 0. 2. 0. 2. 0. 0. 0. 3. 1. 1. 1. 1. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3.
 3. 3. 3. 3. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 3. 4. 4.
 4. 4. 4. 3. 3. 4. 4. 4. 2. 0. 0. 0. 2. 1. 1. 1. 1. 1. 1. 1. 3. 3. 4. 4.
 4. 4. 4. 0. 4. 4. 4. 2. 4. 4. 4. 4. 4. 4. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2

KeyboardInterrupt: 

### 학습 가중치 저장

In [27]:
model.save_parameters("skeletonAR.params")

In [20]:
model.load_parameters("skeletonAR.params", ctx=ctx)

### 검증 데이터로 accuracy 재보기

In [53]:
compare = model(x_valid).argmax(axis=1) == y_valid.squeeze()
compare.sum() / len(compare)


[0.9811765]
<NDArray 1 @gpu(1)>